## Convert RICS DB files into Geojson

In [21]:
import os
import sqlite3
import geojson

# Function to convert a table to GeoJSON
def convert_table_to_geojson(db_path, table_name, output_folder, latitude_column='latitude', longitude_column='longitude'):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Get the list of all columns from the table
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns_info = cursor.fetchall()

        # Extract column names from the table schema
        column_names = [info[1] for info in columns_info]  # info[1] is the column name

        # Form a SQL query to select all columns
        query = f"SELECT {', '.join(column_names)} FROM {table_name};"
        cursor.execute(query)
        

        # Fetch all rows from the table
        rows = cursor.fetchall()

        # Find the index of the latitude and longitude columns
        lat_idx = column_names.index(latitude_column)
        lon_idx = column_names.index(longitude_column)

        # Create a list to hold GeoJSON features
        features = []

        # Loop through the rows and convert each one to a GeoJSON Feature
        for row in rows:
            # Extract the longitude and latitude values for geometry
            point = geojson.Point((row[lon_idx], row[lat_idx]))  # GeoJSON expects (longitude, latitude)

            # Prepare properties dictionary with all other columns
            properties = {}
            for idx, col_name in enumerate(column_names):
                if col_name not in [longitude_column, latitude_column]:  # Exclude spatial columns from properties
                    properties[col_name] = row[idx]

            # Create the GeoJSON feature
            feature = geojson.Feature(geometry=point, properties=properties)
            features.append(feature)

        # Create a GeoJSON FeatureCollection
        feature_collection = geojson.FeatureCollection(features)

        # Write the GeoJSON to a file
        output_file = os.path.join(output_folder, f"{os.path.basename(db_path).replace('.sdb', '')}_{table_name}.geojson")
        with open(output_file, 'w') as f:
            geojson.dump(feature_collection, f)

        print(f"GeoJSON file created for {table_name} in {output_file}")
    except Exception as e:
        print(f"Error processing table {table_name} in {db_path}: {e}")
    finally:
        conn.close()

# Function to find all .sdb files and convert the L and R tables to GeoJSON
def convert_all_sdb_files_to_geojson(folder_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Walk through the folder and find all .sdb files
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".sdb"):
                db_path = os.path.join(root, file)
                print(f"Processing {db_path}...")

                # Convert table 'L' to GeoJSON
                convert_table_to_geojson(db_path, 'L', output_folder)

                # Convert table 'R' to GeoJSON
                convert_table_to_geojson(db_path, 'R', output_folder)

# Folder containing .sdb files and the output folder for GeoJSON files
folder_path = 'RICS_DB'
output_folder = 'RICS_DB_geojson'

# Run the conversion for all .sdb files
convert_all_sdb_files_to_geojson(folder_path, output_folder)


Processing RICS_DB/new_131112_45700.sdb...
GeoJSON file created for L in RICS_DB_geojson/new_131112_45700_L.geojson
GeoJSON file created for R in RICS_DB_geojson/new_131112_45700_R.geojson
Processing RICS_DB/middle_res_131112_24238.sdb...
GeoJSON file created for L in RICS_DB_geojson/middle_res_131112_24238_L.geojson
GeoJSON file created for R in RICS_DB_geojson/middle_res_131112_24238_R.geojson
Processing RICS_DB/cbd_141112_11404.sdb...
GeoJSON file created for L in RICS_DB_geojson/cbd_141112_11404_L.geojson
GeoJSON file created for R in RICS_DB_geojson/cbd_141112_11404_R.geojson
Processing RICS_DB/Industrial_131112_04706.sdb...
GeoJSON file created for L in RICS_DB_geojson/Industrial_131112_04706_L.geojson
GeoJSON file created for R in RICS_DB_geojson/Industrial_131112_04706_R.geojson
Processing RICS_DB/v_new3_131112_43919.sdb...
GeoJSON file created for L in RICS_DB_geojson/v_new3_131112_43919_L.geojson
GeoJSON file created for R in RICS_DB_geojson/v_new3_131112_43919_R.geojson
Proc